In [ ]:
%run ../commons/imports_legal.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
create_database('blp_dataset')
create_table('products_tmp_amazon')

# == Files config ==
FILE_FOLDER = r"D:\legal_data\marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv"
fdata=pd.read_csv(Path(FILE_FOLDER))
IMG_FOLDER_DEST = r"D:\images\amazon"

In [ ]:
fdata.columns

In [ ]:
# Deleting unwanted columns
del_list = ['Uniq Id', 'Brand Name', 'Asin', 'Upc Ean Code', 'List Price', 'Quantity', 'Technical Details', 
            'Model Number', 'Shipping Weight', 'Product Dimensions', 'Variants', 'Sku', 'Product Url', 'Stock', 
            'Product Details', 'Dimensions', 'Color', 'Ingredients', 'Direction To Use', 'Is Amazon Seller', 
            'Size Quantity Variant', 'Product Description']

fdata = fdata.drop(del_list, axis = 1)

fdata.columns

In [ ]:
def update_image(product_id, value):
    query = "UPDATE products_tmp_amazon SET has_image = %s WHERE id = %s"
    cursor.execute(query, (value, product_id))
    conn.commit()

In [ ]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()
    
    return clear_content

In [ ]:
def get_ext(url):
    filename, file_extension = os.path.splitext(url)
    
    return file_extension

In [ ]:
def insert_product(row):
    price = "" if len(row['Selling Price']) > 50 else row['Selling Price']
    
    encoded_string = row['About Product'].encode('utf-8', "ignore")
    description = encoded_string.decode()
    
    description = description.replace("Make sure this fits by entering your model number. | ", "")
    description = re.sub('[👍🌍📱✅【】👐🔬⚗️⚙️🧜😎🔴🔵🙌🌿🎁🦄💃👜⭐💦]', '', description)
    description = clear_text(description.replace(' | ', ''))

    print(description)
    
    query = "INSERT INTO products_tmp_amazon (name, description, price, market_name, has_image, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['Product Name'], description, price, "Amazon", "no_image", "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [ ]:
for index, row in fdata.iterrows():
    if row['Product Name'] == "NaN" or type(row['Product Name']) is not str: row['Product Name'] = ""
    if row['About Product'] == "NaN" or type(row['About Product']) is not str: row['About Product'] = ""
    if row['Selling Price'] == "NaN" or type(row['Selling Price']) is not str: row['Selling Price'] = ""

    if row['Product Name'] is not "" and row['About Product'] is not "":
        product_id = insert_product(row)

        images = row['Image'].split(sep="|")
        img_index = 0

        for img in images:
            if not img == "https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/transparent-pixel.jpg":
                img_ext = get_ext(img)

                folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}{img_ext}")

                if img_index > 0:
                    folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}_{img_index}{img_ext}")

                try:
                    file = urllib.request.urlretrieve(img, folder_dest)
                    img_index = img_index + 1
                except:
                    print(f"Product {product_id}: Error retrieving image ({img})")

                if img_index > 0 : update_image(product_id, "has_image")